### Importing the level data.

In [0]:
# Import required libraries.
import requests as rq
import pandas as pd
import os

In [0]:
#API URL to get water level stations on River Parrett.
api_url = "http://environment.data.gov.uk/hydrology/id/stations.json"
params = {"observedProperty": "waterLevel", "riverName": "River Parrett"}

#Request data from the API using 'get'.
response = rq.get(api_url, params=params)
response.raise_for_status()  # Stop if there's an error and defines the error.
stations = response.json().get("items", []) #Converts the response into a Python dictionary and gets the "items" key.

#Extract relevant data into a list using list comprehension, checking it is a dictionary and is complete. 
station_data = [{"Station Name": station.get("label"),"GUID": station.get("stationGuid"),"Latitude": station.get("lat"),"Longitude": station.get("long"),"River Served": station.get("riverName", "N/A"),"Catchment": station.get("catchmentName", "N/A")} for station in stations if isinstance(station, dict) and station.get("notation")]

#Convert to DataFrame and save as CSV.
df_stations = pd.DataFrame(station_data)

In [0]:
#Initialise a list.
readings = []

#Loop over unique station GUIDs
for guid in df_stations["GUID"].dropna().unique():
    #API URL.
    url = "http://environment.data.gov.uk/hydrology/data/readings.json"
    #Parameters for the request. 
    params = {"station": guid,"observedProperty": "waterLevel","periodName": "15min","mineq-date": "2019-12-19","max-date": "2024-11-25","_limit": 2000000}
    #Parameters for the request. 
    response = rq.get(url, params=params)
    response.raise_for_status()  # Stop if there's an error and defines the error.
    items = response.json().get("items", []) #Returns a Python dictionary of the items.
    # Append readings to the list, adding the GUID for reference.
    for item in items:
        item["GUID"] = guid
        readings.append(item)

#Create a DataFrame from the collected readings.
df_readings = pd.DataFrame(readings)


In [0]:
#Investigate how many unique GUIDs (and therefore stations) there are. 
print(df_readings['GUID'].nunique())

2


In [0]:
df_readings

,measure,date,dateTime,value,quality,GUID,qcode
0,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T00:00:00,2.078,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b,NaN
1,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T00:15:00,2.107,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b,NaN
2,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T00:30:00,2.148,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b,NaN
3,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T00:45:00,2.198,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b,NaN
4,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T01:00:00,2.236,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b,NaN
...,...,...,...,...,...,...,...
346171,{'@id': 'http://environment.data.gov.uk/hydrol...,2024-11-19,2024-11-19T01:00:00,1.719,Good,ce5176cf-1523-4dbe-855f-17a0efffbe02,NaN
346172,{'@id': 'http://environment.data.gov.uk/hydrol...,2024-11-20,2024-11-20T12:45:00,2.270,Good,ce5176cf-1523-4dbe-855f-17a0efffbe02,NaN
346173,{'@id': 'http://environment.data.gov.uk/hydrol...,2024-11-21,2024-11-21T08:45:00,3.924,Good,ce5176cf-1523-4dbe-855f-17a0efffbe02,NaN
346174,{'@id': 'http://environment.data.gov.uk/hydrol...,2024-11-23,2024-11-23T06:45:00,1.450,Good,ce5176cf-1523-4dbe-855f-17a0efffbe02,NaN


In [0]:
#Rename GUID to station.
df_readings.rename(columns={"GUID": "station"}, inplace=True)

In [0]:

os.makedirs("Data", exist_ok=True)
df_readings.to_csv((os.path.join(output_dir, "Imported_API_Level.csv")), index=False)
print(f"CSV saved to {os.path.abspath(output_path)}")


CSV saved to /Workspace/Users/luke.male@environment-agency.gov.uk/Reduced Scope Flooding Work/*** ML Flood Prediction/API Imports and Data/Flood Warning Predictor/Data/Imported_API_Level.csv
